# Introdução

In [ ]:
import os
import re
import time
import random
import tabula
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup

## Functions

In [ ]:
from unicodedata import normalize

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

# Relatório

## Convert PDF

In [ ]:
# URL
url = 'http://www.mpsp.mp.br/portal/page/portal/Promotorias_de_Justica/regioes_adm/com_relacao_comarcas_municipios/Rela%C3%A7%C3%A3o%20munic%C3%ADpios%2013-01-2021%20%20(1).pdf'

# File
file = os.path.join('docs', 'Relação municípios 13-01-2021  (1).pdf')

# Request
r = requests.get(url)
with open(file, 'wb') as f:
    f.write(r.content)

In [ ]:
# Convert PDF into CSV
tabula.convert_into(
    file,
    os.path.join('data', 'tabs', 'tabula.csv'),
    output_format='csv',
    pages='all',
)

In [ ]:
# File
file_in = os.path.join('data', 'tabs', 'tabula.csv')
file_out = os.path.join('data', 'tabs', 'tabula_out.csv')

# Open Files
f_in = open(file_in, 'r')
f_out = open(file_out,'w')
for line in f_in:
	f_out.write(line.replace(',,', ','))

f_in.close()
f_out.close()

In [ ]:
# Read CSV
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tabula_out.csv'),
    usecols=['Localidade', 'Promotoria de Justiça Pertencente', 'Area Regional']
)

# Remove Acentos
df['loc_id'] = df['Localidade'].apply(lambda x: x.replace('MOJI MIRIM', 'Mogi Mirim'))

# Set Lower
df['loc_id'] = df['loc_id'].str.lower()

# Remove Acentos
df['loc_id'] = df['loc_id'].apply(lambda x: remover_acentos(x))



# Remove Acentos
df['pj_id'] = df['Promotoria de Justiça Pertencente'].apply(lambda x: str(x).replace('MOJI MIRIM', 'Mogi Mirim'))

# Set Lower
df['pj_id'] = df['pj_id'].str.lower()

# Remove Acentos
df['pj_id'] = df['pj_id'].apply(lambda x: remover_acentos(x))



# Remove Acentos
df['ar_id'] = df['Area Regional'].apply(lambda x: str(x).replace('MOJI MIRIM', 'Mogi Mirim'))

# Set Lower
df['ar_id'] = df['ar_id'].str.lower()

# Remove Acentos
df['ar_id'] = df['ar_id'].apply(lambda x: remover_acentos(x))

# Deleta Linhas e Colunas
df.drop(
    ['Localidade', 'Promotoria de Justiça Pertencente', 'Area Regional'],
    axis=1,
    inplace=True,
    errors='ignore'
)
df = df.loc[df['loc_id'] != '41.262.199']
df

In [ ]:
# Remove
os.remove(file_in)
os.remove(file_out)

## Municípios

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Set Lower
df_mun['nome_id'] = df_mun['municipio_nome'].str.lower()

# Remove Acentos
df_mun['nome_id'] = df_mun['nome_id'].apply(lambda x: remover_acentos(x))

# Renomeia colunas
df_mun.rename(columns={'municipio_nome': 'NOME'}, inplace=True, errors='ignore')

df_mun

In [ ]:
# Merge Municípios
df = pd.merge(
    df,
    df_mun,    
    how='right',
    left_on='loc_id',
    right_on='nome_id',    
)

# Adjusts
df['loc_id'] = df['NOME']
df['id_loc'] = df['id_municipio']
df.drop(['id_municipio', 'NOME', 'nome_id'], axis=1, inplace=True, errors='ignore')

# Renomeia colunas
df.rename(
    columns={
        'loc_id': 'municipio_nome',
        'id_loc': 'id_municipio',
    },
    inplace=True,
    errors='ignore',
)

# 
df = df[[
    'id_municipio',
    'municipio_nome',
    'pj_id',
    'ar_id',    
]].copy()

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio.csv'),
    index=False,
)
df.head()

## Área Regional

In [ ]:
# AREA REGIONAL
df = pd.read_csv(os.path.join('data', 'tabs', 'tab_municipio.csv'))
df_ar = df[['id_municipio', 'municipio_nome', 'ar_id']].copy()

# Merge
df = pd.merge(
    df_ar,
    df_mun,    
    how='left',
    left_on='ar_id',
    right_on='nome_id',    
)

# Missing Values
dict_ar = {
    'capital': 1,
    'grande sao paulo': 2,
    'vale do ribeira': 3,
}
df_missing = pd.DataFrame(
    list(dict_ar.items()),
    columns=['ar_id','id_ar']
)

# Merge
df = pd.merge(
    df,
    df_missing,    
    how='left',
    left_on='ar_id',
    right_on='ar_id',    
)
df['id_municipio_y'] = df['id_municipio_y'].fillna(0)
df['id_ar'] = df['id_ar'].fillna(0)
df['id_ar'] = df['id_ar'] + df['id_municipio_y']
df['id_ar'] = df['id_ar'].astype(int)


# Renomeia colunas
df.rename(
    columns={
        'id_municipio_x': 'id_municipio',        
        'NOME': 'ar_nome',
    },
    inplace=True,
    errors='ignore'
)

# Deleta
df.drop(
    [
        'ar_id',
        'nome_id',
        'id_municipio_y',
    ],
    axis=1,
    inplace=True,
    errors='ignore'
)

# Missing Values
dict_ar = {
    1: 'Capital',
    2: 'Grande São Paulo',
    3: 'Vale do Ribeira',
}
df_missing = pd.DataFrame(
    list(dict_ar.items()),
    columns=['id_ar', 'NOME']
)
# Merge
df = pd.merge(
    df,
    df_missing,    
    how='left',
    left_on='id_ar',
    right_on='id_ar',    
)
df['ar_nome'] = df['ar_nome'].fillna('') + df['NOME'].fillna('')
df.drop(['NOME'], axis=1, inplace=True, errors='ignore')
df = df[['id_municipio', 'municipio_nome', 'id_ar', 'ar_nome']].copy()

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_ar.csv'),
    index=False,
)
display(df[44:48])

## Área Regional: Group

In [ ]:
# Group
df = pd.DataFrame(df.groupby(['id_ar', 'ar_nome'], dropna=False)['id_municipio'].count())
df = df.reset_index()
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_ar.csv'),
    index=False,
)
df

## Promotorias de Justiça

In [ ]:
def rename_nome(x):
    dict_rename = {
        'embu guacu': 'embu-guacu',
        'estrela doeste': "estrela d'oeste",
        'palmeira doeste': "palmeira d'oeste",
        'sao luiz do paraitinga': 'sao luis do paraitinga',
        'santa rosa do viterbo': 'santa rosa de viterbo',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)    
    return x

In [ ]:
# PRMOTORIAS DE JUSTIÇA
df = pd.read_csv(os.path.join('data', 'tabs', 'tab_municipio.csv'))
df_pj = df[['id_municipio', 'municipio_nome', 'pj_id']].copy()

df_pj.loc[:, 'pj_id'] = df_pj['pj_id'].astype(str).apply(lambda x: rename_nome(x))

# Merge
df = pd.merge(
    df_pj,
    df_mun,    
    how='left',
    left_on='pj_id',
    right_on='nome_id',    
)

#Renomeia colunas
df.rename(
    columns={
        'id_municipio_x': 'id_municipio',
        'id_municipio_y': 'id_pj',
        'NOME': 'pj_nome',
    },
    inplace=True,
    errors='ignore'
)

# Deleta
df.drop(
    ['pj_id', 'nome_id'],
    axis=1,
    inplace=True,
    errors='ignore'
)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_municipio_pj.csv'),
    index=False,
)
df.head()

## Promotorias de Justiça: Group

In [ ]:
# Group
df = pd.DataFrame(df.groupby(['id_pj', 'pj_nome'], dropna=False)['id_municipio'].count())
df = df.reset_index()
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_pj.csv'),
    index=False,
)
df

# Áreas Regionais

In [ ]:
# BS4
url = 'http://www.mpsp.mp.br/portal/page/portal/Promotorias_de_Justica'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
soup = soup.find('map', {'name': 'image-mapa'})

# Create Empty Lists
list_ar = []
list_href = []

# Fill Lists
for i in list(soup.children):
    try:
        #print(i['title'])
        #print(i['href'])
        list_ar.append(str(i['title']))
        list_href.append(str(i['href']))
    except:
        pass

# Assign data to tuples.
list_df = list(zip(list_ar, list_href))

# Converting lists of tuples into
df = pd.DataFrame(list_df, columns = ['ar_nome', 'url'])

# Adjust URLs
prefix = 'http://www.mpsp.mp.br'
df['url'] = df['url'].apply(lambda x: x.replace(prefix, ''))
df['url'] = prefix + df['url']

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_ar_url.csv'),
    index=False,
)

# Promotorias de Justiça

https://docs.google.com/spreadsheets/d/1MuzCoK1uAf_gVqJ6Coi4a1KAEtnYT4Kzu6mTAhZlE_s/edit#gid=0



## Municípios com PJ

In [ ]:
def get_mun_pj(url, ar):
    # BS4
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    soup = soup.find('map', {'name': 'FPMap0'})

    # Create Empty Lists
    list_ar = []
    list_href = []

    # Fill Lists
    for i in list(soup.children):
        try:
            #print(i['alt'])
            #print(i['href'])
            list_ar.append(str(i['alt']))
            list_href.append(str(i['href']))
        except:
            pass

    # Assign data to tuples.
    list_df = list(zip(list_ar, list_href))

    # Converting lists of tuples into
    df = pd.DataFrame(list_df, columns = ['pj', 'url'])

    # Adjust URLs
    prefix = 'http://www.mpsp.mp.br'
    df['url'] = df['url'].apply(lambda x: x.replace(prefix, ''))
    df['url'] = prefix + df['url']

    # Remove Broken Lines
    df['pj'] = df['pj'].apply(lambda x: x.replace('\r\n',''))
    df['url'] = df['url'].apply(lambda x: x.replace('\r\n',''))
    
    # Add Column
    df['municipio'] = df['pj']
    df['ar'] = ar
    df['tem_pj'] = 'Sim'

    # Drop Duplicates and Sort
    df = df.drop_duplicates()
    df = df.sort_values(by=['municipio'], ignore_index=True)

    return df

In [ ]:
# Read AR dataframe
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_ar_url.csv'),
)

# Create Empty dataframe
df_pj_final = pd.DataFrame()

# Loop to append data
for k, v in df.iterrows():
    url = v['url']
    ar = v['ar_nome']
    print(ar)
    try:
        df_pj = get_mun_pj(url, ar)
        df_pj_final = df_pj_final.append(df_pj)
    except:
        pass

# Results
df_pj_final.to_csv(
    os.path.join('data', 'tabs', 'tab_pjs_mun.csv'),
    index=False,
)

## Municípios sem PJ

In [ ]:
def get_mun_small(url, ar):
    # BS4
    #url = 'http://www.mpsp.mp.br/portal/page/portal/Promotorias_de_Justica/regioes_adm/relacoes_regionais/rel_regionais_bauru/mapa_bauru'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    soup = soup.find('div', {'class': 'cabecalho_regiao'})
    soup

    # Sobe dois tables
    soup = soup.find_parent('table')
    soup = soup.find_parent('table')

    # Vai pro proximo table
    soup = soup.find_next('table')
    soup = soup.find_next('table')

    # Table inside table
    soup = soup.find('table')

    data = []
    rows = soup.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        url = row.find('a').get('href')
        cols = [ele.text.strip() for ele in cols]
        cols.append(url)
        data.append([ele for ele in cols if ele])

    # Create DataFrame
    df = pd.DataFrame(
        data,
        columns=['municipio', 'pj', 'url']
    )

    # Adjust URLs
    prefix = 'http://www.mpsp.mp.br'
    df['url'] = df['url'].apply(lambda x: x.replace(prefix, ''))
    df['url'] = prefix + df['url']

    # Remove Broken Lines
    df['municipio'] = df['municipio'].apply(lambda x: x.replace('\r\n',''))
    df['pj'] = df['pj'].apply(lambda x: x.replace('\r\n',''))
    df['url'] = df['url'].apply(lambda x: x.replace('\r\n',''))

    # Adjust
    df['pj'] = df['pj'].apply(lambda x: x.replace('Promotoria de Justiça de ',''))
    df['ar'] = ar
    df['tem_pj'] = 'Não'

    # Drop Duplicates and Sort
    df = df.drop_duplicates()
    df = df.sort_values(by=['municipio'], ignore_index=True)

    # Results
    return df

In [ ]:
# Read AR dataframe
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_ar_url.csv'),
)

# Create Empty dataframe
df_pj_final = pd.DataFrame()

# Loop to append data
for k, v in df.iterrows():
    url = v['url']
    ar = v['ar_nome']
    print(ar)
    try:
        df_pj = get_mun_small(url, ar)
        df_pj_final = df_pj_final.append(df_pj)
    except:
        pass

# Results
df_pj_final.to_csv(
    os.path.join('data', 'tabs', 'tab_pjs_smallmun.csv'),
    index=False,
)

## Merge

In [ ]:
# Results
file = os.path.join('data', 'tabs', 'tab_mun_small.csv')
df_mun_small = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_mun_small.csv'),
)
#os.remove(file)

# Results
file = os.path.join('data', 'tabs', 'tab_mun_pjs.csv')
df_mun_pjs = pd.read_csv(
    file,
)
#os.remove(file)

# Append
df_mun_small = df_mun_small.append(df_mun_pjs)
df = df_mun_small

# Adjust
df = df.sort_values(by=['municipio', 'tem_pj'], ignore_index=True)
df.drop_duplicates(subset='municipio', keep='first', inplace=True)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_pjs.csv'),
    index=False,
)

## Join Municípios

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipios.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Set Lower
df_mun['municipio_nome_lower'] = df_mun['municipio_nome'].str.lower()

# Remove Acentos
df_mun['municipio_nome_lower'] = df_mun['municipio_nome_lower'].apply(lambda x: remover_acentos(x))

In [ ]:
# Results
df = pd.read_csv(
    os.path.join('data', 'tabs', 'tab_pjs.csv'),
)

# Set Lower
df['municipio_nome_lower'] = df['municipio'].str.lower()

# Remove Acentos
df['municipio_nome_lower'] = df['municipio_nome_lower'].apply(lambda x: remover_acentos(x))

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='municipio_nome_lower',
    right_on='municipio_nome_lower'
)

# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_pjs_merge.csv'),
    index=False,
)

# GAEMAs

In [ ]:


for row in soup.find_all('tr'):
    print(row.text)
    #print(row.find('td'))
    try:
        #print(row.get_atrribute('href'))
        pass
    except:
        pass

# Macroregiões

In [ ]:
'http://www.mpsp.mp.br/portal/page/portal/noticias/noticia?id_noticia=19968257&id_grupo=118'